## LSH: Using cosine-similarity

In [2]:
import numpy as np

def generate_hyperplanes(dim, num_hash_functions):
    """
    Generate random hyperplanes for hash functions.
    
    Parameters:
    - dim: Dimensionality of the embeddings.
    - num_hash_functions: Number of hash functions per table.
    
    Returns:
    - A matrix of shape (num_hash_functions, dim) where each row is a hyperplane.
    """
    return np.random.randn(num_hash_functions, dim)

def hash_vectors(vectors, hyperplanes):
    """
    Hash a batch of vectors using a set of hyperplanes.

    Parameters:
    - vectors: Input vectors (2D array of shape [n_samples, d]).
    - hyperplanes: Matrix of hyperplanes (2D array of shape [k, d]).

    Returns:
    - A matrix of binary hash values (shape [n_samples, k]).
    """
    # Compute dot products and return binary hash values
    return (np.dot(vectors, hyperplanes.T) > 0).astype(int)

In [3]:
### Test hashing ###

np.random.seed(42)
num_vectors =  int(1e4) #int(10) # guess we'll have around 2 million beer reviews?
dim = 768
num_hash_functions = 15

# Generate random vectors and hyperplanes
test_vectors = np.random.randn(num_vectors, dim)  # 5 vectors with 768 dimensions
test_hyperplanes = np.random.randn(num_hash_functions, dim)  # 10 hyperplanes

# Test the hash_vectors function
hash_values = hash_vectors(test_vectors, test_hyperplanes)
hash_values

# Takes about 52 seconds to hash 2 million vectors with 768 dimensions using 15 hash functions

hash_keys = [tuple(h) for h in hash_values]



In [4]:
# Build LSH framework
from collections import defaultdict

class LSHVectorized:
    def __init__(self, d, k, L):
        """
        Initialize the LSH scheme with vectorized support.

        Parameters:
        - d: Dimensionality of the input vectors.
        - k: Number of hash functions per table.
        - L: Number of hash tables.
        """
        self.L = L
        self.tables = [defaultdict(list) for _ in range(L)]
        self.hyperplanes = [generate_hyperplanes(d, k) for _ in range(L)]

    def add_vectors(self, vectors, identifiers):
        """
        Add a batch of vectors to the LSH index.

        Parameters:
        - vectors: Input vectors (2D array of shape [n_samples, d]).
        - identifiers: A list of unique identifiers for the vectors.
        """
        for table, hyperplanes in zip(self.tables, self.hyperplanes):
            # Compute hash values for all vectors at once
            hash_values = hash_vectors(vectors, hyperplanes)
            
            # Convert binary hash values to tuples for dictionary keys
            hash_keys = [tuple(h) for h in hash_values]
            
            # Add vectors to their corresponding buckets
            for identifier, key in zip(identifiers, hash_keys):
                table[key].append(identifier)

    def query(self, vectors):
        """
        Query the LSH index to find similar items for a batch of vectors.

        Parameters:
        - vectors: Query vectors (2D array of shape [n_samples, d]).

        Returns:
        - A list of sets, where each set contains the candidates for a query vector.
        """
        candidates = [set() for _ in range(len(vectors))]
        for table, hyperplanes in zip(self.tables, self.hyperplanes):
            # Compute hash values for all query vectors
            hash_values = hash_vectors(vectors, hyperplanes)
            
            # Convert binary hash values to tuples for dictionary keys
            hash_keys = [tuple(h) for h in hash_values]
            
            # Retrieve candidates for each query
            for i, key in enumerate(hash_keys):
                candidates[i].update(table.get(key, []))
        return candidates


In [12]:
### Test LSH framework ###
# Parameters
d = 768  # Dimensionality of BERT embeddings
k = 10   # Number of hash functions per table
L = 5    # Number of hash tables

# Initialize vectorized LSH
lsh = LSHVectorized(d, k, L)

# Generate random embeddings and IDs
np.random.seed(42)
embeddings = np.random.randn(100, d)  # 100 random embeddings
ids = [f"beer_{i}" for i in range(100)]  # Unique IDs

# Add all embeddings to the LSH index in one batch
lsh.add_vectors(embeddings, ids)

# Query multiple embeddings
query_embeddings = embeddings[:5]  # First 5 embeddings as queries
candidates = lsh.query(query_embeddings)

# Print candidates for each query
for i, query_candidates in enumerate(candidates):
    print(f"Candidates for query {i}: {query_candidates}")


Candidates for query 0: {'beer_0'}
Candidates for query 1: {'beer_1'}
Candidates for query 2: {'beer_62', 'beer_2'}
Candidates for query 3: {'beer_84', 'beer_72', 'beer_3'}
Candidates for query 4: {'beer_99', 'beer_4'}
